# Imports

In [ ]:
import os
import pandas as pd

# Download and load the data

In [ ]:
import kagglehub
path = kagglehub.dataset_download("ishika9bhatia/power-bi-sales-dashboard-online-sales-analysis")
print("Path to dataset files:", path)

100%|██████████| 155k/155k [00:00<00:00, 11.1MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/ishika9bhatia/power-bi-sales-dashboard-online-sales-analysis/versions/1


In [ ]:
details_path = os.path.join(path, "Details.csv")
orders_path  = os.path.join(path, "Orders.csv")

In [ ]:
details_df = pd.read_csv(details_path)
orders_df  = pd.read_csv(orders_path)

# Understand the data

In [ ]:
details_df.head(10)

,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode
0,B-25681,1096,658,7,Electronics,Electronic Games,COD
1,B-26055,5729,64,14,Furniture,Chairs,EMI
2,B-25955,2927,146,8,Furniture,Bookcases,EMI
3,B-26093,2847,712,8,Electronics,Printers,Credit Card
4,B-25602,2617,1151,4,Electronics,Phones,Credit Card
5,B-25881,2244,247,4,Clothing,Trousers,Credit Card
6,B-25696,275,-275,4,Clothing,Saree,COD
7,B-25687,387,-213,5,Clothing,Saree,UPI
8,B-25643,50,-44,2,Clothing,Hankerchief,UPI
9,B-25851,135,-54,5,Clothing,Kurti,COD


In [ ]:
orders_df.head(10)

,Order ID,Order Date,CustomerName,State,City
0,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura
1,B-25993,03-02-2018,Madhav,Delhi,Delhi
2,B-25973,24-01-2018,Madan Mohan,Uttar Pradesh,Mathura
3,B-25923,27-12-2018,Gopal,Maharashtra,Mumbai
4,B-25757,21-08-2018,Vishakha,Madhya Pradesh,Indore
5,B-25967,21-01-2018,Sudevi,Uttar Pradesh,Prayagraj
6,B-25955,16-01-2018,Shiva,Maharashtra,Pune
7,B-26093,27-03-2018,Sarita,Maharashtra,Pune
8,B-25798,01-10-2018,Shishu,Andhra Pradesh,Hyderabad
9,B-25602,01-04-2018,Vrinda,Maharashtra,Pune


In [ ]:
details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      1500 non-null   object
 1   Amount        1500 non-null   int64 
 2   Profit        1500 non-null   int64 
 3   Quantity      1500 non-null   int64 
 4   Category      1500 non-null   object
 5   Sub-Category  1500 non-null   object
 6   PaymentMode   1500 non-null   object
dtypes: int64(3), object(4)
memory usage: 82.2+ KB


In [ ]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      500 non-null    object
 1   Order Date    500 non-null    object
 2   CustomerName  500 non-null    object
 3   State         500 non-null    object
 4   City          500 non-null    object
dtypes: object(5)
memory usage: 19.7+ KB


In [ ]:
print(details_df.duplicated().sum())

0


In [ ]:
print(orders_df.duplicated().sum())

0


In [ ]:
details_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Amount,1500.0,291.847333,461.924620,4.0,47.75,122.0,326.25,5729.0
Profit,1500.0,24.642000,168.558810,-1981.0,-12.00,8.0,38.00,1864.0
Quantity,1500.0,3.743333,2.184942,1.0,2.00,3.0,5.00,14.0


In [ ]:
orders_df.describe().T

,count,unique,top,freq
Order ID,500,500,B-26095,1
Order Date,500,307,24-11-2018,7
CustomerName,500,336,Shreya,6
State,500,19,Maharashtra,94
City,500,25,Indore,71


# Clean text

In [ ]:
for col in details_df.select_dtypes(include="object"):
    details_df[col] = details_df[col].str.strip()
for col in orders_df.select_dtypes(include="object"):
    orders_df[col] = orders_df[col].str.strip()

# Convert dates

In [ ]:
orders_df["Order Date"] = pd.to_datetime(orders_df["Order Date"],
                                       format="%d-%m-%Y", errors="coerce")

# Merge the details and order files

In [ ]:
df = details_df.merge(orders_df, on="Order ID", how="inner")

# Adding some neccessary columns for the dashboard

In [ ]:
df["Year"] = df["Order Date"].dt.year
df["Month"] = df["Order Date"].dt.month
df["YearMonth"] = df["Order Date"].dt.to_period("M").astype(str)
df["ProfitMargin"] = (df["Profit"] / df["Amount"] * 100).round(2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Order ID      1500 non-null   object        
 1   Amount        1500 non-null   int64         
 2   Profit        1500 non-null   int64         
 3   Quantity      1500 non-null   int64         
 4   Category      1500 non-null   object        
 5   Sub-Category  1500 non-null   object        
 6   PaymentMode   1500 non-null   object        
 7   Order Date    1500 non-null   datetime64[ns]
 8   CustomerName  1500 non-null   object        
 9   State         1500 non-null   object        
 10  City          1500 non-null   object        
 11  Year          1500 non-null   int32         
 12  Month         1500 non-null   int32         
 13  YearMonth     1500 non-null   object        
 14  ProfitMargin  1500 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int3

# Reordering the columns

In [ ]:
df = df[[
    "Order ID", "Order Date", "Year", "Month", "YearMonth",
    "CustomerName", "State", "City",
    "Category", "Sub-Category", "PaymentMode",
    "Quantity", "Amount", "Profit", "ProfitMargin"
]]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Order ID      1500 non-null   object        
 1   Order Date    1500 non-null   datetime64[ns]
 2   Year          1500 non-null   int32         
 3   Month         1500 non-null   int32         
 4   YearMonth     1500 non-null   object        
 5   CustomerName  1500 non-null   object        
 6   State         1500 non-null   object        
 7   City          1500 non-null   object        
 8   Category      1500 non-null   object        
 9   Sub-Category  1500 non-null   object        
 10  PaymentMode   1500 non-null   object        
 11  Quantity      1500 non-null   int64         
 12  Amount        1500 non-null   int64         
 13  Profit        1500 non-null   int64         
 14  ProfitMargin  1500 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int3